![../docs/images/banner.png](../docs/images/banner.png)


# [Contact Us](mailto:info@inverted.ai) for the server ip address.


In [ ]:
! pip install iai-client
from IPython.display import clear_output
clear_output()

In [ ]:
from iai_client.utils import Res, SensorSettings, Resolution, PyGameWindow, ClientSideBoundingBoxes
from iai_client.interface import IAIEnv
import numpy as np
import time
import cv2
from IPython.display import clear_output
import matplotlib.pyplot as plt
import pygame
import argparse
from IPython.display import clear_output
clear_output(wait=True)

In [ ]:
def_res = Res.MD
scale = 1

sensors_dict = {
        'top-cam': {
            'sensor_type': 'camera',
            'camera_type': 'rgb-camera',
            'bounding_box': True,
            'track_actor_types': SensorSettings.Available_Tracked_Actors,
            'show_bounding_boxes': True,
            'world_sensor': False,
            'resolution': def_res,
            'location': SensorSettings.Location(x=0.0, z=80.0, y=0.0),
            'rotation': SensorSettings.Rotation(yaw=90.0, pitch=-90.0, roll=0.0),
            },
            'front-cam-seg': {
            'sensor_type': 'camera',
            'camera_type': 'segmentation',
            'bounding_box': True,
            'track_actor_types': SensorSettings.Available_Tracked_Actors,
            'show_bounding_boxes': True,
            'world_sensor': False,
            'resolution': def_res,
            'location': SensorSettings.Location(x=0, z=2.8, y=0),
            'rotation': SensorSettings.Rotation(yaw=0, roll=0, pitch=0),
            'radius': 20,
            },
            'side-cam':{
            'sensor_type': 'camera',
            'camera_type': 'rgb-camera',
            'bounding_box': False,
            # 'track_actor_types': 'all', #Actors, # or 'all'
            'track_actor_types': SensorSettings.Available_Tracked_Actors,
            'show_bounding_boxes': False,
            'world_sensor': False,
            'resolution': def_res,
            'location': SensorSettings.Location(x=0, z=2.8, y=0),
            'rotation': SensorSettings.Rotation(yaw=90, roll=0, pitch=0),
            'fov': 90.0,
            },
        'boundingbox_side_cam': {
            'sensor_type': 'boundingbox',
            'track_actor_types': SensorSettings.Available_Tracked_Actors,  # or 'all'
            'world_sensor': False, # if True returns the coordinates in the global frame of reference
            'attach_to_actor': 'side-cam',
            'radius': 20,
            'occlusion': True,
            },
}

In [ ]:
parser = argparse.ArgumentParser()
IAIEnv.add_config(parser)
config = parser.parse_args(['--client_id', 'mycompany'])
server_ip = input('Enter server IP:')
config.zmq_server_address = f"{server_ip}:5555"
env = IAIEnv(config)
world_parameters = dict(carlatown='Town04')

In [ ]:
obs = env.set_scenario('egodriving', world_parameters=world_parameters, sensors=sensors_dict)

In [ ]:
def jupyter_render(obs):
    fig, ax = plt.subplots()
    plt.cla()
    ax.axis('off')
    plt.imshow(obs['front_image'].astype(int))
    fig, ax = plt.subplots()
    ax.axis('off')
    plt.imshow(obs['birdview_image'].astype(int))
    plt.show()
pygame.init()

dim = (scale*def_res.width, scale*def_res.height)
width = np.sum([sensors_dict[sns]['resolution'].width*scale for sns in sensors_dict if sensors_dict[sns]['sensor_type']=='camera'])
height = np.max([sensors_dict[sns]['resolution'].height*scale for sns in sensors_dict if sensors_dict[sns]['sensor_type']=='camera'])
full_res = Resolution(width, height)
main_display = PyGameWindow(full_res)

In [ ]:
action = [0.0, 0.0]
obs, reward, done, info = env.step(action)
disp_img = np.concatenate(list(cv2.resize(obs['sensor_data'][name]['image'], dim, interpolation = cv2.INTER_AREA) for name in sensors_dict if 'image' in obs['sensor_data'][name].keys()), axis=1)
main_display.render(disp_img)
pygame.display.update()
jupyter_render(obs)
rem_self = 0
howlong = 0
while not done:
    prev_action = obs['prev_action']
    if rem_self < howlong:
        action = info['expert_action']
        obs, reward, done, info = env.step(action)
        rem_self += 1
    else:
        cmd = input('Enter Steering:')
        if cmd == 'reset':
            obs = env.reset()
            jupyter_render(obs)
        elif cmd == 'init':
            obs = env.set_scenario('egodriving', world_parameters=world_parameters, sensors=sensors_dict)
        elif cmd == 'end':
            print(env.end_simulation())
            break
        elif cmd == 'self':
            howlong = int(input('How many steps:'))
            rem_self = 0
        else:
            angle = float(cmd)
            acceleration = float(input('Enter Acceleration:'))
            action = (acceleration, angle)
            obs, reward, done, info = env.step(action)
    clear_output(wait=True)
    jupyter_render(obs)
    for name in sensors_dict:
        if (sensors_dict[name]['sensor_type'] == 'boundingbox'):
            if (sensors_dict[name]['attach_to_actor'] != 'ego'):
                attached_sensor = sensors_dict[name]['attach_to_actor']
                bb2d = ClientSideBoundingBoxes.get_2d_bbox(obs['sensor_data'][name]['bounding_boxes'], sensors_dict[attached_sensor]['location'], sensors_dict[attached_sensor]['rotation'], sensors_dict[attached_sensor]['fov'], sensors_dict[attached_sensor]['resolution'], obs['compact_vector'][:3], obs['compact_vector'][3:6], coordinate_system='attached_sensor', occlusion=True)
                img = obs['sensor_data'][attached_sensor]['image']
                obs['sensor_data'][attached_sensor]['image'] = ClientSideBoundingBoxes.draw_bounding_boxes_on_array(img, bb2d,  draw2d=True, occlusion=True)

    disp_img = np.concatenate(list(cv2.resize(obs['sensor_data'][name]['image'], dim, interpolation = cv2.INTER_AREA) for name in sensors_dict if 'image' in obs['sensor_data'][name].keys()), axis=1)
    main_display.render(disp_img)
    pygame.display.update()
print(f'Episode Done, Reward:{reward}')


In [ ]:
print(env.end_simulation())
env.close()